# Data Perparation

In [2]:
# Import relevant libraries
import dask.dataframe as dd
from datetime import time, timedelta, datetime
import pandas as pd
import numpy as np

In [28]:
# Read in dataset
ddf = dd.read_parquet("../data/0_RAW/parking_ticket_2022.parquet")
ddf.head(5)

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,...,vehicle_color,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping,hydrant_violation,double_parking_violation
0,1457617912,JEB5683,NY,PAS,2021-06-25T00:00:00.000,40,VAN,FORD,P,63430,...,GY,0,2007,-,3,NaN,NaN,NaN,NaN,NaN
1,1457617924,JAN2986,NY,PAS,2021-06-25T00:00:00.000,20,SUBN,DODGE,P,13490,...,BLU,0,2007,-,0,NaN,NaN,NaN,NaN,NaN
2,1457622427,FJH6630,TX,PAS,2021-06-17T00:00:00.000,98,SDN,AUDI,P,79430,...,WHITE,0,0,-,0,NaN,NaN,NaN,NaN,NaN
3,1457638629,RD1Y5N,MO,PAS,2021-06-16T00:00:00.000,98,SDN,TOYOT,P,53130,...,TAN,0,2001,-,0,NaN,NaN,NaN,NaN,NaN
4,1457639580,T503814C,NY,OMT,2021-07-04T00:00:00.000,40,TAXI,HONDA,P,81030,...,WHI,0,2020,-,0,NaN,NaN,NaN,NaN,NaN


In [22]:
len(ddf.columns)

43

In [1]:
ddf.plate_id

NameError: name 'ddf' is not defined

In [29]:
# Define column types
ddf = ddf.astype(dtype={'summons_number':'int32', 'violation_code':'int16'})

In [30]:
# Create violation_datetime
ddf['issue_date'] = dd.to_datetime(ddf['issue_date'])

## Handling missing values

Control if these columns are missing some important attributes.
- 'summons_number'
- 'plate_id'
- 'registration_state'
- 'plate_type',
- 'issue_date'
- 'violation_code'
- 'vehicle_body_type', 'vehicle_make',
       'issuing_agency', 'street_code1', 'street_code2', 'street_code3',
       'vehicle_expiration_date', 'violation_location', 'violation_precinct',
       'issuer_precinct', 'issuer_code', 'issuer_command', 'issuer_squad',
       'violation_time', 'time_first_observed', 'violation_county',
       'violation_in_front_of_or', 'house_number', 'street_name',
       'intersecting_street', 'date_first_observed', 'law_section',
       'sub_division', 'violation_legal_code', 'days_parking_in_effect',
       'from_hours_in_effect', 'to_hours_in_effect', 'vehicle_color',
       'unregistered_vehicle', 'vehicle_year', 'meter_number',
       'feet_from_curb', 'violation_post_code', 'violation_description',
       'no_standing_or_stopping', 'hydrant_violation',
       'double_parking_violation']

In [25]:
ddf['violation_post_code'].isnull().compute()

0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Name: violation_post_code, Length: 1000, dtype: bool

In [26]:
ddf.dropna(subset=['violation_post_code'], how='any').compute()

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,...,vehicle_color,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping,hydrant_violation,double_parking_violation


## Handling Outliers

In [31]:
ddf.drop(ddf[ddf['datetime'] > datetime.now()].index, axis=0)

KeyError: 'datetime'

In [32]:
def get_datetime(row):
    time = row['violation_time']
    date = row['issue_date']
    min = int(time[2:4])
    """
    if time == 'foo':
        hour = np.nan
        min = np.nan"""
    if time[-1] == 'P':
        hour = int(time[:2]) + 12 % 24
    else:
        hour = int(time[:2])
    datetime = date + timedelta(hours=hour, minutes=min)
    return datetime

In [33]:
ddf['datetime'] = ddf.apply(get_datetime, axis=1, meta=str)

/home/yannik/anaconda3/envs/bigdata/lib/python3.9/site-packages/dask/dataframe/core.py:7067: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


In [34]:
ddf.compute()

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,...,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping,hydrant_violation,double_parking_violation,datetime
0,1457617912,JEB5683,NY,PAS,2021-06-25,40,VAN,FORD,P,63430,...,0,2007,-,3,NaN,NaN,NaN,NaN,NaN,2021-06-25 01:30:00
1,1457617924,JAN2986,NY,PAS,2021-06-25,20,SUBN,DODGE,P,13490,...,0,2007,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-25 02:25:00
2,1457622427,FJH6630,TX,PAS,2021-06-17,98,SDN,AUDI,P,79430,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-17 20:09:00
3,1457638629,RD1Y5N,MO,PAS,2021-06-16,98,SDN,TOYOT,P,53130,...,0,2001,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-16 18:05:00
4,1457639580,T503814C,NY,OMT,2021-07-04,40,TAXI,HONDA,P,81030,...,0,2020,-,0,NaN,NaN,NaN,NaN,NaN,2021-07-04 22:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1457412469,60352ML,NY,COM,2021-06-30,14,VAN,DODGE,P,0,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-30 12:35:00
996,1466155048,NGHJ60,FL,PAS,2021-06-21,17,SDN,VOLKS,X,75530,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 07:10:00
997,1466155061,C68NSB,NJ,PAS,2021-06-21,17,SDN,TOYOT,X,75530,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 07:15:00
998,1466155670,FKR5645,NC,PAS,2021-06-21,17,SDN,BMW,C,40390,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 11:23:00


In [ ]:
ddf['datetime'].head()

0   2021-06-25 01:30:00
1   2021-06-25 02:25:00
2   2021-06-17 20:09:00
3   2021-06-16 18:05:00
4   2021-07-04 22:58:00
Name: datetime, dtype: datetime64[ns]

In [35]:
ddf['datetime'] = dd.to_datetime(ddf['datetime'])

In [36]:
ddf.compute()

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,...,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping,hydrant_violation,double_parking_violation,datetime
0,1457617912,JEB5683,NY,PAS,2021-06-25,40,VAN,FORD,P,63430,...,0,2007,-,3,NaN,NaN,NaN,NaN,NaN,2021-06-25 01:30:00
1,1457617924,JAN2986,NY,PAS,2021-06-25,20,SUBN,DODGE,P,13490,...,0,2007,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-25 02:25:00
2,1457622427,FJH6630,TX,PAS,2021-06-17,98,SDN,AUDI,P,79430,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-17 20:09:00
3,1457638629,RD1Y5N,MO,PAS,2021-06-16,98,SDN,TOYOT,P,53130,...,0,2001,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-16 18:05:00
4,1457639580,T503814C,NY,OMT,2021-07-04,40,TAXI,HONDA,P,81030,...,0,2020,-,0,NaN,NaN,NaN,NaN,NaN,2021-07-04 22:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1457412469,60352ML,NY,COM,2021-06-30,14,VAN,DODGE,P,0,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-30 12:35:00
996,1466155048,NGHJ60,FL,PAS,2021-06-21,17,SDN,VOLKS,X,75530,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 07:10:00
997,1466155061,C68NSB,NJ,PAS,2021-06-21,17,SDN,TOYOT,X,75530,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 07:15:00
998,1466155670,FKR5645,NC,PAS,2021-06-21,17,SDN,BMW,C,40390,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2021-06-21 11:23:00


In [37]:
ddf = ddf.drop(['issue_date', 'violation_time'], axis=1).compute()

In [19]:
ddf[ddf['datetime'] > datetime.now()]

,summons_number,plate_id,registration_state,plate_type,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,street_code2,...,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping,hydrant_violation,double_parking_violation,datetime
593,1472434640,ELZ938,MN,PAS,40,SDN,NISSA,P,18090,7790,...,0,0,-,0,NaN,NaN,NaN,NaN,NaN,2031-07-01 23:09:00


In [40]:
ddf = ddf.drop(ddf[ddf['datetime'] > datetime.now()].index, axis=0)

In [41]:
# export dataset
ddf.to_parquet("../data/1_PREPARED/parking_ticket_2022.parquet")

## Prepare Weather Data

In [37]:
df_weather = dd.read_parquet("../data/0_RAW/weather_2022.parquet")
df_weather.head()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2022-01-01 00:00:00,8.9,8.4,97.0,0.0,NaN,192.0,7.6,NaN,1014.4,NaN,4.0
2022-01-01 01:00:00,8.6,8.3,98.0,0.0,NaN,205.0,12.6,NaN,1014.0,NaN,4.0
2022-01-01 02:00:00,8.6,8.1,97.0,0.0,NaN,202.0,11.5,NaN,1013.6,NaN,5.0
2022-01-01 03:00:00,8.0,7.7,98.0,0.0,NaN,200.0,10.4,NaN,1013.7,NaN,4.0
2022-01-01 04:00:00,7.7,7.4,98.0,0.0,NaN,205.0,8.6,NaN,1013.6,NaN,4.0


In [38]:
df_weather.compute().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8737 entries, 2022-01-01 00:00:00 to 2022-12-31 00:00:00
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   temp    8737 non-null   float64
 1   dwpt    8737 non-null   float64
 2   rhum    8737 non-null   float64
 3   prcp    8737 non-null   float64
 4   snow    0 non-null      float64
 5   wdir    8737 non-null   float64
 6   wspd    8737 non-null   float64
 7   wpgt    0 non-null      float64
 8   pres    8737 non-null   float64
 9   tsun    0 non-null      float64
 10  coco    8505 non-null   float64
dtypes: float64(11)
memory usage: 819.1 KB


### Check for null values

In [40]:
# As snow, wpgt and tsun only have null values we drop them
# For coco we use forward fill to fill the null values

df_weather = df_weather.drop(['snow', 'wpgt', 'tsun'], axis=1)

df_weather['coco'] = df_weather['coco'].ffill()

In [41]:
df_weather.isna().sum().compute()

temp    0
dwpt    0
rhum    0
prcp    0
wdir    0
wspd    0
pres    0
coco    0
dtype: int64

### Feature engineering for Weather Condition Code

In [44]:
# Add string feature for weather condition TODO
weather_condition = {
'1':'Clear',
'2':'Fair',
'3':'Cloudy',
'4':'Overcast',
"5":'Fog',
"6":'Freezing Fog',
"7":'Light Rain',
"8":'Rain',
"9":'Heavy Rain',
"10":'Freezing Rain',
"11":'Heavy Freezing Rain',
"12":'Sleet',
"13":'Heavy Sleet',
"14":'Light Snowfall',
"15":'Snowfall',
"16":'Heavy Snowfall',
"17":'Rain Shower',
"18":'Heavy Rain Shower',
"19":'Sleet Shower',
"20":'Heavy Sleet Shower',
"21":'Snow Shower',
"22":'Heavy Snow Shower',
"23":'Lightning',
"24":'Hail',
"25":'Thunderstorm',
"26":'Heavy Thunderstorm',
"27":'Storm'}

In [47]:
df_weather = df_weather.astype(dtype={'coco':'int8'})

In [50]:
#df_weather['condition'] = weather_condition[df_weather['coco'].compute()]

TypeError: unhashable type: 'Series'